In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 

pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_tensor_type(torch.DoubleTensor)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1


## read data 

- for each different neurons numbers, 32, 64, ..., 512, we run the fixed-size iterative OGA for 10 trials and take the average errors. 
- The initial error is the result from a random feature method
- The final error is the result after iterations of the fixed-size iterative OGA. 

In [16]:
def target(x):
    z = torch.cos(4 * pi *x[:,0:1]) * torch.cos(2 * pi *x[:,1:2])
    return z 
function_name = 'cos4pix1cos2pix2'
load_oga = False 
load_init = False 
model_none = False
plot = False 
save_data = True 
relu_k = 1 

start_time = time.time() 
exponent = 9 
neuron_num_list = np.array([2**ii for ii in range(5,exponent + 1)])
num_trials = 10  
err_trials = torch.zeros(len(neuron_num_list),num_trials)
err_arr_rfm = np.zeros(len(neuron_num_list))
err_arr_iter = np.zeros(len(neuron_num_list))
for i, num_neurons in enumerate(neuron_num_list):
    print("neuron number: ",num_neurons)
    errl2_trials = []
    for trial in range(num_trials):
        num_epochs = num_neurons * 2  ###  
        for N1 in [2**7]: 
            s = 2**2
            N = s * N1 
            nn_size = num_neurons    
            plot = False 
            plot_freq = num_epochs 
            Nx = 100 
            order = 3 

            folder_name = './'

            filename = folder_name + 'err_{}_N_{}_neuron_{}_epochs_{}_randomized_{}.pt'.format(function_name,s*N, num_neurons,num_epochs,trial)
            errl2 = torch.load(filename)
            errl2_trials.append(errl2)
    errl2_trials = torch.stack(errl2_trials,dim = 0)
    errl2_mean = errl2_trials.mean(dim = 0)
    err_arr_rfm[i] = errl2_mean[0].item()
    err_arr_iter[i] = errl2_mean[-1].item()
    print("{:.2e}, {:.2e}".format(errl2_mean[0].item(),errl2_mean[-1].item()) )    

#compute convergence order 

print("rfm order")
print(-np.log(err_arr_rfm[:-1] / err_arr_rfm[1:])/ np.log(neuron_num_list[:-1]/neuron_num_list[1:]))

print("iterative oga order")
print(-np.log(err_arr_iter[:-1] / err_arr_iter[1:]) /np.log(neuron_num_list[:-1]/neuron_num_list[1:]))


neuron number:  32
4.56e-01, 7.92e-02
neuron number:  64
3.44e-01, 3.40e-02
neuron number:  128
2.21e-01, 1.40e-02
neuron number:  256
1.05e-01, 5.45e-03
neuron number:  512
4.40e-02, 2.14e-03
rfm order
[0.40692734 0.6387245  1.07300443 1.2537814 ]
iterative oga order
[1.2211548  1.28127841 1.35859806 1.3460522 ]


In [11]:
print("rfm order")
print(np.log(err_arr_rfm[:-1] / err_arr_rfm[1:])/ np.log(neuron_num_list[:-1]/neuron_num_list[1:]))

# print("iterative oga order")
# print(np.log(err_arr_iter[:-1] / err_arr_iter[1:]) /np.log(neuron_num_list[:-1]/neuron_num_list[1:]))

rfm order
[nan nan nan nan]


/tmp/ipykernel_2298014/1551838532.py:2: RuntimeWarning: invalid value encountered in true_divide
  print(np.log(err_arr_rfm[:-1] / err_arr_rfm[1:])/ np.log(neuron_num_list[:-1]/neuron_num_list[1:]))


In [12]:
neuron_num_list

array([ 32,  64, 128, 256, 512])

In [13]:
err_arr_rfm

array([0., 0., 0., 0., 0.])